# Proyek Prediksi Risiko Diabetes dengan Random Forest

Notebook ini bertujuan untuk membangun model Machine Learning yang dapat memprediksi risiko diabetes dengan akurasi yang **SEIMBANG** antara Training dan Testing (Mencegah Overfitting).

### Langkah Utama:
1. **Imputasi Data**: Mengganti nilai 0 yang tidak logis pada kolom medis dengan nilai rata-rata.
2. **Pemisahan Data**: Memisahkan fitur (X) dan target (Y).
3. **Standarisasi**: Menyamakan skala data.
4. **Modeling**: Menggunakan **Random Forest Classifier** dengan parameter khusus agar model tidak 'menghapal' data (Overfitting).

## 1. Import Library
Kita mengimpor library penting untuk pengolahan data (`pandas`, `numpy`) dan machine learning (`sklearn`).

In [40]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## 2. Memuat Dataset
Membaca file `diabetes.csv` ke dalam DataFrame Pandas. Pastikan file ini sudah diunggah di direktori yang sama.

In [41]:
dataset_path = 'diabetes.csv'
diabetes_dataset = pd.read_csv(dataset_path)

print(f"Ukuran data awal: {diabetes_dataset.shape}")
diabetes_dataset.head()

Ukuran data awal: (768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## 3. Preprocessing: Handling Missing Values

Banyak kolom medis seperti `Glucose`, `BloodPressure`, `SkinThickness`, `Insulin`, dan `BMI` memiliki nilai **0**, yang secara medis tidak mungkin (invalid).

**Strategi Perbaikan:**
1. Ubah nilai `0` menjadi `NaN` (Not a Number).
2. Isi `NaN` tersebut dengan nilai **Rata-rata (Mean)** dari masing-masing kolom.

In [42]:
cols_to_fix = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# 1. Ganti 0 dengan NaN
diabetes_dataset[cols_to_fix] = diabetes_dataset[cols_to_fix].replace(0, np.nan)

# 2. Isi NaN dengan Mean (Rata-rata)
for col in cols_to_fix:
    mean_val = diabetes_dataset[col].mean(skipna=True)
    diabetes_dataset[col] = diabetes_dataset[col].fillna(mean_val)
    print(f"Mengisi nilai kosong di kolom '{col}' dengan rata-rata: {mean_val:.2f}")

print("\nCek data setelah imputasi:")
diabetes_dataset.head()

Mengisi nilai kosong di kolom 'Glucose' dengan rata-rata: 121.69
Mengisi nilai kosong di kolom 'BloodPressure' dengan rata-rata: 72.41
Mengisi nilai kosong di kolom 'SkinThickness' dengan rata-rata: 29.15
Mengisi nilai kosong di kolom 'Insulin' dengan rata-rata: 155.55
Mengisi nilai kosong di kolom 'BMI' dengan rata-rata: 32.46

Cek data setelah imputasi:


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.00000,155.548223,33.6,0.627,50,1
1,1,85.0,66.0,29.00000,155.548223,26.6,0.351,31,0
2,8,183.0,64.0,29.15342,155.548223,23.3,0.672,32,1
3,1,89.0,66.0,23.00000,94.000000,28.1,0.167,21,0
4,0,137.0,40.0,35.00000,168.000000,43.1,2.288,33,1


## 4. Pemisahan Fitur (X) dan Target (Y)

Tahap ini krusial. Kita harus memisahkan antara data yang menjadi **petunjuk** (Features) dan data yang menjadi **kunci jawaban** (Target).

### Penjelasan Code:
- `X = diabetes_dataset.drop(columns='Outcome', axis=1)`:
  - Kita mengambil semua kolom **kecuali** 'Outcome'.
  - Data ini (Pregnancies, Glucose, dst) akan menjadi input bagi model untuk belajar memprediksi.
  - `axis=1` menandakan kita membuang kolom (bukan baris).

- `Y = diabetes_dataset['Outcome']`:
  - Kita hanya mengambil kolom 'Outcome'.
  - Ini adalah label yang berisi **0 (Sehat)** atau **1 (Diabetes)**.
  - Model akan mencoba mencocokkan pola di X agar sesuai dengan Y.

In [43]:
X = diabetes_dataset.drop(columns='Outcome', axis=1)
Y = diabetes_dataset['Outcome']

print("X (Fitur):")
print(X.head())
print("\nY (Target):")
print(Y.head())

X (Fitur):
   Pregnancies  Glucose  BloodPressure  SkinThickness     Insulin   BMI  \
0            6    148.0           72.0       35.00000  155.548223  33.6   
1            1     85.0           66.0       29.00000  155.548223  26.6   
2            8    183.0           64.0       29.15342  155.548223  23.3   
3            1     89.0           66.0       23.00000   94.000000  28.1   
4            0    137.0           40.0       35.00000  168.000000  43.1   

   DiabetesPedigreeFunction  Age  
0                     0.627   50  
1                     0.351   31  
2                     0.672   32  
3                     0.167   21  
4                     2.288   33  

Y (Target):
0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64


## 5. Standarisasi Data

Algoritma Machine Learning bekerja lebih optimal jika semua fitur memiliki skala yang seragam (rata-rata 0, deviasi standar 1). Kita menggunakan `StandardScaler`.

In [44]:
scaler = StandardScaler()
scaler.fit(X)
standarized_data = scaler.transform(X)

X = standarized_data
print("Data setelah standarisasi (5 baris pertama):")
print(X[:5])

Data setelah standarisasi (5 baris pertama):
[[ 6.39947260e-01  8.65108070e-01 -3.35182392e-02  6.65502121e-01
  -3.34507888e-16  1.66291742e-01  4.68491977e-01  1.42599540e+00]
 [-8.44885053e-01 -1.20616153e+00 -5.29859033e-01 -1.74633816e-02
  -3.34507888e-16 -8.52531177e-01 -3.65060778e-01 -1.90671905e-01]
 [ 1.23388019e+00  2.01581340e+00 -6.95305964e-01  8.08793628e-16
  -3.34507888e-16 -1.33283341e+00  6.04397318e-01 -1.05584152e-01]
 [-8.44885053e-01 -1.07465235e+00 -5.29859033e-01 -7.00428884e-01
  -7.24388738e-01 -6.34211980e-01 -9.20762614e-01 -1.04154944e+00]
 [-1.14185152e+00  5.03457822e-01 -2.68066914e+00  6.65502121e-01
   1.46550563e-01  1.54897999e+00  5.48490910e+00 -2.04963989e-02]]


## 6. Train-Test Split

Membagi dataset menjadi:
- **Training Set (80%)**: Untuk melatih model.
- **Testing Set (20%)**: Untuk menguji seberapa pintar model kita pada data baru.

`stratify=Y` digunakan agar perbandingan jumlah pasien diabetes dan non-diabetes tetap seimbang di kedua set.

In [45]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2, stratify=Y)

print(f"Jumlah Data Training: {X_train.shape[0]}")
print(f"Jumlah Data Testing : {X_test.shape[0]}")

Jumlah Data Training: 614
Jumlah Data Testing : 154


## 7. Melatih Model (Random Forest)

Untuk mencapai **Akurasi Training 100%**, kita menggunakan **Random Forest Classifier**.

### Mengapa Random Forest?
- Random Forest terdiri dari banyak *Decision Trees* (Pohon Keputusan).
- Algoritma ini sangat powerful. Namun jika tidak dibatasi, ia cenderung **Overfitting** (Hafal data training sampai 100% tapi jelek saat ujian/testing).
- Oleh karena itu, kita akan membatasi kedalaman pohon (`max_depth`) dan minimal data per daun (`min_samples_leaf`) agar model belajar pola umum saja, bukan menghafal.

In [46]:
# n_estimators=100 artinya kita menggunakan 100 pohon keputusan.
# Tuning Hyperparameter untuk Mencegah Overfitting
# max_depth=5, min_samples_leaf=5 digunakan agar model tidak terlalu kompleks (mencegah hafalan/overfitting)
classifier = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_leaf=5, random_state=42)
classifier.fit(X_train, Y_train)

RandomForestClassifier(max_depth=5, min_samples_leaf=5, random_state=42)

## 8. Evaluasi Akurasi

Kita cek akurasi model pada data training dan testing.

In [47]:
# Evaluasi Training Data
X_train_prediction = classifier.predict(X_train)
train_acc = accuracy_score(X_train_prediction, Y_train)
print(f'Akurasi Training: {train_acc * 100:.2f}%')

# Evaluasi Testing Data
X_test_prediction = classifier.predict(X_test)
test_acc = accuracy_score(X_test_prediction, Y_test)
print(f'Akurasi Testing : {test_acc * 100:.2f}%')

Akurasi Training: 83.22%
Akurasi Testing : 74.68%


## 9. Simulasi Prediksi
Mencoba memprediksi satu pasien baru.

In [48]:
input_data = (5, 166, 72, 19, 175, 25.8, 0.587, 51)

# Ubah ke DataFrame agar sesuai dengan fitur saat training
feature_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
input_df = pd.DataFrame([input_data], columns=feature_names)

# Standarisasi
std_data = scaler.transform(input_df)

# Prediksi
prediction = classifier.predict(std_data)

print(f"Data Input: {input_data}")
if prediction[0] == 0:
    print('Hasil: Pasien TIDAK Berisiko Diabetes')
else:
    print('Hasil: Pasien BERISIKO Diabetes')

Data Input: (5, 166, 72, 19, 175, 25.8, 0.587, 51)
Hasil: Pasien BERISIKO Diabetes
